In [3]:
import nltk
import pprint
import numpy as np
import pandas as pd
import os
import urllib
import re
import codecs
from random import shuffle
from sklearn.cross_validation import train_test_split
from shutil import copyfile
from tqdm import *
from PIL import Image
pp = pprint.PrettyPrinter(indent=2, depth=2, width=80)

os.chdir("C:\TalkingData\PackageName Combine")

C:\Anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Original Data

In [9]:
sample = pd.read_csv(".\data\packageName.csv",
                     index_col = False,
                     names = ['app_store','pkg_name','app_des','app_id','app_name','create_time','detail_url','developer',
                              'download_times','hash','id','img_url','parent_class_id','parent_class_name','pkg_volume','plat',
                              'release_time','score','sub_class','sub_class_id','tag','update_date_time','version'])

In [12]:
print(sample.shape)
sample[0:5]

(97473, 23)


,app_store,pkg_name,app_des,app_id,app_name,create_time,detail_url,developer,download_times,hash,...,parent_class_name,pkg_volume,plat,release_time,score,sub_class,sub_class_id,tag,update_date_time,version
0,360手机助手,by.game.binumbers,*容易上瘾的手机版2048游戏，好玩到根本就停不下来！快来挑战*强大脑！玩法简单：只要移动数...,1594547.0,2048,2017-09-18 09:47:26.0,http://openbox.mobilem.360.cn/detail/index/sof...,作者：Alexander Kulikovski,1034万次,1501289555253090889,...,单机游戏,5.88M,Android,更新时间：2014-09-12,8.9,NaN,NaN,消除 卡通 触屏 单机 休闲益智,2017-09-18 10:01:00.0,版本：2.7
1,360手机助手,com.ahajoy.foodDefense,世界美食和塔防的完美结合 在游戏中牵动你的味觉神经 游戏介绍： 美食厨防，舌尖上的塔防来了！...,2634761.0,美食厨防,2017-09-18 09:39:42.0,http://openbox.mobilem.360.cn/detail/index/sof...,作者：广州哈卓互联网科技有限公司,2万次,7798690187786870956,...,塔防策略,25.32M,Android,更新时间：2015-12-09,7.8,NaN,NaN,塔防 策略塔防,2017-09-18 12:02:15.0,版本：1.16.0
2,360手机助手,com.appsrainbow.SubwaySimulator4BerlinUBahnEdi...,你知道做一个火车司机是什么感觉？你对付黑暗的地铁隧道和长转弯，控制庞大的机器 --- 地铁，...,3308343.0,地铁模拟器 - 柏林版,2017-09-18 11:09:34.0,http://openbox.mobilem.360.cn/detail/index/sof...,作者：北京乐虹科技有限公司,3万次,-6344758532962789595,...,模拟经营,88.92M,Android,更新时间：2016-06-30,6.0,NaN,NaN,NaN,2017-09-18 11:23:10.0,版本：1.0
3,360手机助手,com.batovi.pixelcupsoccer2,像素世界杯16(Pixel Cup Soccer 16)是一款像素风足球游戏，游戏的玩法十分...,3446047.0,像素世界杯16,2017-09-18 09:52:18.0,http://openbox.mobilem.360.cn/detail/index/sof...,作者：Panic Art Studios,8万次,6332718646293212854,...,小编精选,58.42M,Android,更新时间：2016-08-24,6.4,NaN,NaN,像素 小编精选 竞技 足球 竞技游戏,2017-09-18 12:25:54.0,版本：1.0.1
4,360手机助手,com.bigpinwheel.game.runfast,"跑得快打一副牌时，也称为""关牌""，主要流行于江浙一带，大学校园当中亦很盛行。 【玩法：...",151481.0,跑得快,2017-09-18 10:01:39.0,http://openbox.mobilem.360.cn/detail/index/sof...,作者：大风车工作室,149万次,2246562418762300821,...,棋牌天地,4.05M,Android,更新时间：2014-08-13,5.9,NaN,NaN,棋牌 跑得快 纸牌 单机,2017-09-18 10:14:48.0,版本：1.1110


## App Store (Source of data)

In [154]:
# App stores (Where the data comes from)
sample.app_store.unique()

array(['360手机助手', '小米应用商店', '应用汇', '快用', '拇指玩', '豌豆荚'], dtype=object)

## Package Name

In [156]:
num_unique_pkgName = len(sample.pkg_name.unique())
num_records = sample.shape[0]
print('Total of {} records have {} duplicated packageName values'.format(num_records,num_records-num_unique_pkgName))

Total of 97473 records have 3141 duplicated packageName values


In [11]:
sample.pkg_name.unique()[0:100]

array(['by.game.binumbers', 'com.ahajoy.foodDefense',
       'com.appsrainbow.SubwaySimulator4BerlinUBahnEdition',
       'com.batovi.pixelcupsoccer2', 'com.bigpinwheel.game.runfast',
       'com.caohua.wddhh.qihoo', 'com.chyo.fleet.p360',
       'com.cocos2d.fishingfun.qihoo', 'com.dddwan.summer.fondant',
       'com.djinnworks.StickmanDownhill', 'com.echo.pixeldungeon',
       'com.feelingtouch.sniperzombie', 'com.funcity.carsbattle',
       'com.gamedo.hlw.qihoo', 'com.GF.PalaceM2_cn_cn.hwy_android_360',
       'com.happyelements.AndroidClover.www', 'com.hoge.akazukin_2',
       'com.hxjy.union.qihoo', 'com.istorm.kdyg.qihoo',
       'com.jjh.qihoo.Fish', 'com.joym.ultraman.shouji360',
       'com.kairogame.android.Frontier.qihoo', 'com.knight.union.qihoo',
       'com.lemon.play.majiang', 'com.ltgame.hjlmzbz.c360',
       'com.mars.fktzdzjtcl', 'com.Moyoufang.Stickmanbadminton',
       'com.netease.hit.qihoo', 'com.netease.ssy.qihoo',
       'com.noodlecake.tryharder.free', 'com.ou

## App Description

1. Have nan values (Might also contains 'null'/'n/a'/empty etc.)
    - Need to remove them before doing topic modeling or something
2. Not all apps have app description
3. Should consider removing meaningless information such as 系统，语言，更新时间 and etc.
4. Try jieba (Unigram, Bigram, Trigram and etc.)
5. LDA 提取不同主题的关键词 -> 分类模型特征

In [159]:
print(sample.app_des[0])

*容易上瘾的手机版2048游戏，好玩到根本就停不下来！快来挑战*强大脑！玩法简单：只要移动数字方块将相同的数字合并在一起，分数便会增加。你能成功达到2048吗？这不太容易！尽情享受，赢得胜利！ 游戏特色： - 色彩丰富的图形动画效果，令您赏心悦目 - 多种游戏模式：2048、4096、8192和无尽模式 - 保存成果：上一次的游戏资料会被保存，玩家可继续玩下一关 - 夜间模式 - 统计资料 - 在同一台设备上可创建不同玩家帐户 - 自动保存 - 平板电脑体验*优化，支持横屏模式 - 完美的音效 - 独特的设计 - 支持多国语言（包括中文） 以上描述中存在违反广告法的内容，自动用*代替 【基本信息】 作者：Alexander Kulikovski 更新时间：2014-09-12 版本：2.7 系统：Android 2.3以上 语言：中文


In [160]:
print(sample.app_des[1])

世界美食和塔防的完美结合 在游戏中牵动你的味觉神经 游戏介绍： 美食厨防，舌尖上的塔防来了！ 这款游戏将塔防与美食烹饪结合，玩家在享受游戏快乐的同时，还能体验世界各地不同的美食文化。 游戏背景： 为了成为*的厨师，你将踏上挑战美食大赛的旅途，利用塔防厨具修炼各种菜谱，但是凡事都不会一帆风顺，小心那些变异食材和贪婪的老鼠！ 游戏特点： 1、各大菜系的美食在游戏中一一呈现，指尖舌尖全都停不下来。 2、炮塔超级模式，一颗小小的能量豆让你瞬间改变战局。 3、食材的反击，它们可不会老老实实就让你加工了。 4、首创塔防游戏的双BOSS玩法， BOSS也会组队来袭！ 5、主流剧情之外还隐藏有若干特殊玩法关卡。 6、九大神器助你提升战斗厨艺。 7、暗藏彩蛋玩法等你发现。 客服信息 客服 QQ：1483758413 客服电邮：1483758413@qq.com 以上描述中存在违反广告法的内容，自动用*代替 【基本信息】 作者：广州哈卓互联网科技有限公司 更新时间：2015-12-09 版本：1.16.0 系统：Android 2.3以上 语言：中文 【更新内容】 1、降低了部分关卡难度； 2、再次强化了厨具超级模式时的暴击表现； 3、修正了叉子的爆击机率问题； 4、增加了签到奖励数值； 5、优化部分美术效果；


In [169]:
print(sample.app_des[200])

急用钱、借点钱、无抵押现金借款、无征信小额贷款、利率低信用贷款、放款快、机构多、小额贷款APP 小额贷款、现金借钱、信用借贷平台，30分钟极速放款， 500-50000元轻松借！ -小额贷款利息低：借款1万,月还50元,月息低至0.5%,每月还款无压力 -贷款口子更新快：保证大家能找到适合自己的贷款口子 -小额贷款放款快：审核通过快至30分钟放款 -一站式小额贷款服务：我们提供专业的贷款搜索、智能匹配、和申请等服务。 【产品特色】 1、闪电借款：极速小额贷款，线上全自动化审核，快至1小时到账。 2、额度灵活：小额轻松借，大额高至十万！ 3、门槛超低：无抵押无担保小额贷款，有手机号就能借。 4、操作便捷：现金借款进度实时查询，到期分期还款提醒，借还方便。 5、贷款攻略：教您如何贷款，如何提高信用积分，如何提额等 6、小额贷款利息低：综合比较选择适合自己的贷款产品。 7、征信查询：个人征信查询，提供醉新、醉全面的个人征信息查询和贷款服务。 8、社保查询：查询基本养老、基本医疗、生育保险、失业保险、工伤保险等缴费信息； 9、公积金查询：对接全国公积金管理中心，随时随地查询公积金缴存明细、余额；1：增加个人信息中心 2：增加在线客服 3：增加密码登陆 4：优化产品体验度


In [167]:
print(sample.app_des[500])

nan


## App Name

1. Remove punctuations
2. Should we do something with the english characters or completely remove them?
3. Hopefully to get some key words from topic modeling or TF-IDF (Consider doing this separately from app description, because the information in app description might overweight the information in app name)

**Comment**: This feature might be useful for the classification of app category, but not necessarilly for combining packageName to metaID. Since different versions of apps might want to be consider with different metaID. They nearly have the exact same app name except one might have a digit indicating the version (EX. 植物大战僵尸，植物大战僵尸2)

In [8]:
len(sample.app_name.unique())

89720

In [170]:
sample.app_name[0:10]

0                2048
1                美食厨防
2         地铁模拟器 - 柏林版
3             像素世界杯16
4                 跑得快
5          无敌大航海-后宫争霸
6          暴风战舰-巅峰跨服战
7          捕鱼达人4-休闲游戏
8    夏目的美丽日记-恋爱换装手...
9             火柴人山地飙车
Name: app_name, dtype: object

In [171]:
sample.app_name[200:210]

200          口子来了小额贷款
201    全局复制:Universal
202              动漫滤镜
203                潮极
204               云展业
205          IELTSBRO
206         冷冻箱省电优化神器
207       Armorfly浏览器
208        肖邦大冒险2地狱归来
209             老王万年历
Name: app_name, dtype: object

## Image URL

In [5]:
for i in tqdm(range(10)):
    print(sample.img_url[i])

  0%|                                                   | 0/10 [00:00<?, ?it/s]

http://p17.qhimg.com/t01ec2372578902007f.png
http://p19.qhimg.com/t0174b520a381f0c229.png
http://p19.qhimg.com/t016d3b4cb4c877b932.png
http://p15.qhimg.com/t01ce0eca01be6353b3.png
http://p3.qhimg.com/t015a60f671741d72bf.png
http://p15.qhimg.com/t01edd856e8ed3e48da.png
http://p17.qhimg.com/t01803baa31846deb5e.png
http://p15.qhimg.com/t0176fba1d2d6b04367.png
http://p18.qhimg.com/t01d12b1e5fd718e208.png
http://p1.qhimg.com/t01f4c4d3c31db7dca4.png


100%|█████████████████████████████████████████| 10/10 [00:00<00:00, 163.92it/s]


## Detail URL

This is the URL of the source of web scaping (Sometime the URL doesn't exist anymore)

In [177]:
for i in range(10):
    print(sample.detail_url[i])

http://openbox.mobilem.360.cn/detail/index/soft_id/1594547?recrefer=L_G_app_1594547|old
http://openbox.mobilem.360.cn/detail/index/soft_id/2634761?recrefer=L_G_app_2634761|old
http://openbox.mobilem.360.cn/detail/index/soft_id/3308343?recrefer=L_G_app_3308343|old
http://openbox.mobilem.360.cn/detail/index/soft_id/3446047?recrefer=L_G_app_3446047|old
http://openbox.mobilem.360.cn/detail/index/soft_id/151481?recrefer=L_G_app_151481|old
http://openbox.mobilem.360.cn/detail/index/soft_id/3726136?recrefer=L_G_app_3726136|old
http://openbox.mobilem.360.cn/detail/index/soft_id/3304224?recrefer=L_G_app_3304224|old
http://openbox.mobilem.360.cn/detail/index/soft_id/3372511?recrefer=L_G_app_3372511|old
http://openbox.mobilem.360.cn/detail/index/soft_id/3127698?recrefer=L_G_app_3127698|old
http://openbox.mobilem.360.cn/detail/index/soft_id/738645?recrefer=L_G_app_738645|old


## Developer

Can be used to double-check if the combination of package name is reasonable (Two apps from different developer usually shouldn't be combined together)

Several text cleaning steps:
1. Remove meaningless information such as: 
    - 作者:
    - 科技有限公司
    - 工作室 (Studios)
2. Convert to lowercase
3. Remove punctuations

In [178]:
sample.developer[0:10]

0    作者：Alexander Kulikovski
1           作者：广州哈卓互联网科技有限公司
2              作者：北京乐虹科技有限公司
3       作者：Panic Art Studios
4                  作者：大风车工作室
5          作者：湖南草花互动网络科技有限公司
6          作者：深圳市中手游网络科技有限公司
7            作者：天津触控盛世科技有限公司
8            作者：天津英雄互娱科技有限公司
9         作者：Djinnworks e.U.
Name: developer, dtype: object

## Download Times

Can't think of its usefulness at this moment.....

In [183]:
sample.download_times[0:5]

0    1034万次
1       2万次
2       3万次
3       8万次
4     149万次
Name: download_times, dtype: object

## Sub Class

In [186]:
print(len(sample.sub_class.unique()))
sample.sub_class.unique()

78


array([nan, '解谜', '逃脱', '发泄', '战略', '冒险', '第一人称', '音乐', '迷宫', '战术', '海战',
       '生存', '建造', '战棋', '枪战', '回合', '闯关', '其他球类', '飞行', '拼图', '第三人称',
       '消除', '城市', '拉力赛', 'RPG', '塔防', '恋爱', '换装', '养成', '狙击', '即时', '跑酷',
       '扑克', '经营', '直线竞速', '街道赛', '探索', '交通', '家园', '祖玛', '跳跃', '自行车',
       '格斗', '极限运动', '躲避', '赛马', '农场', '桌球', '摩托车', '越野赛', '方块', '弹球',
       '足球', '找茬', '捕鱼', '平衡', '篮球', '答题', '坦克', '水上竞速', '餐厅', 'MOBA',
       '趣味运动', '卡丁车', '麻将', '斗地主', '帝国', '动物', '水上运动', '跳棋', '德州', '数独',
       '围棋', '象棋', '三国杀', '飞行棋', '七巧板', '矿洞'], dtype=object)

## Tag

In [187]:
print(len(sample.tag.unique()))
sample.tag.unique()

6204


array(['消除 卡通 触屏 单机 休闲益智 ', '塔防 策略塔防 ', nan, ...,
       '生活应用 运动 运动健康 出行必用 健身 运动·健身 健身应用 目标 ', '益智休闲 对战 棋牌 黑白 棋类 黑白棋 ',
       '休闲 益智休闲 趣味 消除 战斗 经典 动作 关卡 '], dtype=object)

# Labled Data

In [4]:
sample_standard = pd.read_csv(".\data\sample_standard",
                   header = 0,
                   names = ['metaid','appName_orig','name_en','name_ch','appName_standard','packageName','hash','iconurl','typeid',
                            'parentid','TD_category','developerName','version','update','platform','reference','comment','lastupdate'])

In [5]:
len(sample_standard.metaid.unique())

15502

In [9]:
sample_standard.shape

(19289, 18)

In [10]:
sample_standard[500:510]

,metaid,appName_orig,name_en,name_ch,appName_standard,packageName,hash,iconurl,typeid,parentid,TD_category,developerName,version,update,platform,reference,comment,lastupdate
500,357,消灭星星2016-送话费,NaN,xiaomiexingxing2016-songhuafei,NaN,com.zplay.migupopstar,5395120606622297847,http://img02.muzhiwan.com/2017/05/24/com.zplay...,T201300,T200000,休闲游戏,NaN,v1.3.6,2017-08-04,2,http://www.muzhiwan.com/com.zplay.migupopstar....,NaN,2018/2/26
501,357,消灭星星2016-送话费,NaN,xiaomiexingxing2016-songhuafei,NaN,com.zplay.migupopstar.anzhi,-8577171809196900289,http://pp.myapp.com/ma_icon/0/icon_12187251_14...,T201300,T200000,休闲游戏,NaN,NaN,NaN,2,NaN,NaN,2018/2/26
502,359,铁路12306,NaN,tielu12306,铁路12306,com.MobileTicket,7499170796297973860,http://p7.qhimg.com/t0106019ab0b5744df6.png,T104040,T104000,铁路,中铁程科技有限责任公司,2.8,2017/7/17,2,http://zhushou.360.cn/detail/index/soft_id/903638,NaN,2018/2/26
503,360,消灭星星PopStar,NaN,xiaomiexingxingPopStar,消灭星星PopStar,com.zplay.popstar.zw.vivo,-5572588461193434636,http://pp.myapp.com/ma_icon/0/icon_3167_146124...,T201300,T200000,休闲游戏,NaN,NaN,NaN,2,NaN,NaN,2018/2/26
504,360,消灭星星PopStar,NaN,xiaomiexingxingPopStar,NaN,com.zplay.popstar.zw.oppo,-5572588461209051932,http://pp.myapp.com/ma_icon/0/icon_3167_146124...,T201300,T200000,休闲游戏,NaN,NaN,NaN,2,NaN,NaN,2018/2/26
505,360,消灭星星PopStar,NaN,xiaomiexingxingPopStar,NaN,com.brianbaek.popstar.y2345,7412603283392267567,http://pp.myapp.com/ma_icon/0/icon_3167_146124...,T201300,T200000,休闲游戏,NaN,NaN,NaN,2,NaN,NaN,2018/2/26
506,360,消灭星星PopStar,NaN,xiaomiexingxingPopStar,NaN,com.tencent.tmgp.popstar2sns,-6350444081192499620,http://pp.myapp.com/ma_icon/0/icon_3167_146124...,T201300,T200000,休闲游戏,NaN,NaN,NaN,2,NaN,NaN,2018/2/26
507,360,消灭星星PopStar,NaN,xiaomiexingxingPopStar,NaN,com.zplay.popstar.zw.zzhd2,7860674533258102910,http://pp.myapp.com/ma_icon/0/icon_3167_146124...,T201300,T200000,休闲游戏,NaN,NaN,NaN,2,NaN,NaN,2018/2/26
508,360,消灭星星PopStar,NaN,xiaomiexingxingPopStar,NaN,com.brianbaek.popstar.youku,7412603283530541013,http://pp.myapp.com/ma_icon/0/icon_3167_146124...,T201300,T200000,休闲游戏,NaN,NaN,NaN,2,NaN,NaN,2018/2/26
509,360,消灭星星PopStar,NaN,xiaomiexingxingPopStar,NaN,com.zplay.popstar.zw.kn,8832313004207806511,http://pp.myapp.com/ma_icon/0/icon_3167_146124...,T201300,T200000,休闲游戏,NaN,NaN,NaN,2,NaN,NaN,2018/2/26


# Download App Icon Images

In [19]:
def download_icon(data, id_col, url_col): # The data must be a pandas dataframe and have two columns: metaid, icon_url
    """
    This function dowlands icon image as .jpg file from URL. Some images can only be downloaded while connect to VPN, which helps
    us access website oversea.

    The target path for downloaded image is: "C:/TalkingData/PackageName Combine/image"

    The folder will contains both **The downlaoded image** as well as **Error_Log.txt**, 
    which records the log of errors for images that failed to download
    """
    
    # Data = sample_standard, id_col = 'metaid', url_col = 'iconurl'
    
    # Sort the data by metaid
    data = data.sort_values(by = [id_col,url_col])
    
    # Initialize previous metaid holder and url holder
    prev_metaid = ''
    prev_url = ''
    # Initialize current metaid holder and url holder
    current_metaid = ''
    current_url = ''
    # Initialize icon counter for the same metaid
    icon_counter = -999

    # Set the working directory to image folder
    output_path = "C:/TalkingData/PackageName Combine/image"
    os.chdir(output_path)    
    
    # If the output folder is NOT empty, empty the folder
    fileList = os.listdir(output_path)
    if fileList:
        keyboard_input = input("The output folder has existed files. Do you want to empty the folder? (y/n)")
        if keyboard_input.lower() == 'y':
            # Empty every file in the folder
            for fileName in fileList:
                os.remove(output_path + "/" + fileName)
            print("\nAll files in the output folder has been wiped out!\n")
        elif keyboard_input.lower() == 'n':
            print("\nTake your time to deal with those files :)")
            return
        else:
            print("\nPlease enter either y or n")
            return
    
    for index in tqdm(range(data.shape[0])):
        
        # Current icon link
        link = data.loc[index,url_col]
        
        # If this is the first record
        if prev_metaid == '':
            current_metaid = data.loc[index,id_col]
            current_url = data.loc[index,url_col]
            prev_metaid = data.loc[index,id_col]
            prev_url = data.loc[index,url_col]
            icon_counter = 1
        else:            
            current_metaid = data.loc[index,id_col]
            current_url = data.loc[index,url_col]
            # If the pointer move to a new app (based on metaid)
            if prev_metaid != current_metaid:
                # Re-assign the value to prev_metaid
                prev_metaid = current_metaid
                prev_url = current_url
                # Re-initialize the counter
                icon_counter = 1
            # If the pointer still points to records belongs to the same app (based on metaid)
            else:
                # If the url is not the same
                if prev_url != current_url:
                    # Increment the counter
                    icon_counter += 1
                else:
                    next
            
        # Current file name
        filename = r"{}_{}.jpg".format(current_metaid,icon_counter)
        
        # Retrive the image from URL and save it as JPG
        try:
            urllib.request.urlretrieve(link, filename)
        # Write the Error Log to txt file
        except urllib.error.URLError as err:
            with codecs.open('Error_Log.txt','a','utf-8') as f:
                f.write('Index: {}, Link: {}\n URL Error, Reason: {}\n'.format(index, link, err.reason))
            f.close()
            continue
        except urllib.error.HTTPError as err: # e.g. HTTPError 404
            with codecs.open('Error_Log.txt','a','utf-8') as f:
                f.write('Index: {}, Link: {}\n HTTP Error Code {}, Reason: {}\n'.format(index, link, err.code, err.reason))
            f.close()
            continue
        except ConnectionResetError as err:
            with codecs.open('Error_Log.txt','a','utf-8') as f:
                f.write('Index: {}, Link: {}\n ConnectionResetError:{}\n'.format(index, link, err))
            f.close()
            continue
        except ValueError as err:
            with codecs.open('Error_Log.txt','a','utf-8') as f:
                f.write('Index: {}, Link: {}\n ValueError: {}\n'.format(index, link, err))
            f.close()
            continue
        except TypeError as err: # e.g. nan
            with codecs.open('Error_Log.txt','a','utf-8') as f:
                f.write('Index: {}, Link: {}\n TypeError: {}\n'.format(index, link, err))
            f.close()
            continue

In [ ]:
# Download image from url
# download_icon(sample_standard,'metaid','iconurl')

The output folder has existed files. Do you want to empty the folder? (y/n)y

All files in the output folder has been wiped out!



 52%|██████████████████▋                 | 10046/19289 [15:00<13:48, 11.16it/s]

# Check label error

In [46]:
sample_standard[(sample_standard.metaid==5578)|(sample_standard.metaid==117308)]

,metaid,appName_orig,name_en,name_ch,appName_standard,packageName,hash,iconurl,typeid,parentid,TD_category,developerName,version,update,platform,reference,comment,lastupdate
4299,5578,一起欢乐斗地主,NaN,yiqihuanledoudizhu,一起欢乐斗地主,com.bohaoo.doudizhu.HUAWEI,-3069078904196073845,http://pp.myapp.com/ma_icon/0/icon_10078435_14...,T202000,T200000,棋牌游戏,NaN,NaN,NaN,2,NaN,NaN,2018/2/26
4300,5578,一起欢乐斗地主,NaN,yiqihuanledoudizhu,NaN,com.bohaoo.guanwanbaidu,5687855777121502948,http://pp.myapp.com/ma_icon/0/icon_10078435_14...,T202000,T200000,棋牌游戏,NaN,NaN,NaN,2,NaN,NaN,2018/2/26
4301,5578,一起欢乐斗地主,NaN,yiqihuanledoudizhu,NaN,com.bohaoo.hldoudizhu.mi,59326790103357944,http://file.market.xiaomi.com/thumbnail/PNG/l1...,T202000,T200000,棋牌游戏,上海域起网络科技有限公司,4.0.1.0,2017-08-18,2,http://app.mi.com/details?id=com.bohaoo.hldoud...,NaN,2018/2/26
4302,5578,一起欢乐斗地主,NaN,yiqihuanledoudizhu,NaN,com.bohaoo.doudizhu,-7843105176209930326,http://pp.myapp.com/ma_icon/0/icon_10078435_14...,T202000,T200000,棋牌游戏,NaN,NaN,NaN,2,NaN,NaN,2018/2/26
16081,117308,斗地主,NaN,doudizhu,斗地主,cn.xlgame.xlddzsa.baidu,4855603287310664227,http://pp.myapp.com/ma_icon/0/icon_11994659_14...,T202000,T200000,棋牌游戏,NaN,NaN,NaN,2,NaN,NaN,2018/2/26
16082,117308,斗地主,NaN,doudizhu,NaN,com.danjidzz.doudizhu,-1880164722943170628,http://pp.myapp.com/ma_icon/0/icon_11994659_14...,T202000,T200000,棋牌游戏,NaN,NaN,NaN,2,NaN,NaN,2018/2/26


In [48]:
sample_standard[sample_standard.metaid==14747]

,metaid,appName_orig,name_en,name_ch,appName_standard,packageName,hash,iconurl,typeid,parentid,TD_category,developerName,version,update,platform,reference,comment,lastupdate
8364,14747,密室逃脱5逃出博物馆,NaN,mishitaotuo5taochubowuguan,密室逃脱5逃出博物馆,com.martian.RoomEscape5.ex,6506285767612899972,http://p15.qhimg.com/t01515d30a416452fe2.png,T201400,T200000,智力游戏,深圳市火星人互动娱乐有限公司,5.17.41,2017/5/8,2,http://zhushou.360.cn/detail/index/soft_id/318...,NaN,2018/2/26
8365,14747,密室逃脱3之逃出办公室,NaN,mishitaotuo3zhitaochubangongshi,NaN,com.martian.RoomEscape8.nearme.gamecenter,-3081267826966829169,http://pp.myapp.com/ma_icon/0/icon_12207685_14...,T201400,T200000,智力游戏,NaN,NaN,NaN,2,NaN,NaN,2018/2/26
8366,14747,密室逃脱3之逃出办公室,NaN,mishitaotuo3zhitaochubangongshi,NaN,com.martian.RoomEscape3.gfan,-3771820236094291357,http://pp.myapp.com/ma_icon/0/icon_12207685_14...,T201400,T200000,智力游戏,NaN,NaN,NaN,2,NaN,NaN,2018/2/26
8367,14747,密室逃脱3之逃出办公室,NaN,mishitaotuo3zhitaochubangongshi,NaN,com.martian.RoomEscape3.ex,6506285767608403790,http://p15.qhimg.com/t015fccc7dbfd87c917.png,T201400,T200000,智力游戏,深圳市火星人互动娱乐有限公司,3.0.2,2016/3/10,2,http://zhushou.360.cn/detail/index/soft_id/314...,NaN,2018/2/26
8368,14747,密室逃脱3之逃出办公室,NaN,mishitaotuo3zhitaochubangongshi,NaN,com.martian.RoomEscape3.oppo,-3771820236076133053,http://pp.myapp.com/ma_icon/0/icon_12207685_14...,T201400,T200000,智力游戏,NaN,NaN,NaN,2,NaN,NaN,2018/2/26
